In [1]:
import nbdev

In [2]:
#| default_exp core

In [3]:
#|export
from __future__ import annotations
from typing import List, Callable, TypeVar,  Generic, Sequence, Union, Optional, Any, Set, Tuple, Dict, Protocol
from dataclasses import dataclass

### Types

In [4]:
#| export

T = TypeVar("T")
TEvent = TypeVar('TEvent', bound=object) 
THandler = Callable[[TEvent], None] 
TSubscriber = Callable[[T], None] # action to invoke when the store changes
TUnsubscriber = Callable[[], None] # action to invoke upon graceful termination of the subscription
noop = lambda: None
TObserver = Callable[[TEvent], None]
class TObservable(Protocol):
    def subscribe(self, subscriber: TSubscriber) -> TUnsubscriber: ...
TSubject = Union[TObservable, TObserver]

In [37]:
class Base:
    def __str__(self) -> str: return repr(self)
    def __repr__(self) -> str: return f"{self.__class__.__name__}({str(self.__dict__)})"
    
@dataclass
class Event:
    source: str
    type: str
    payload: Any = None
    
    def __init__(self, source: str, **attrs: Any) -> None:
        self.source = source
        for k, v in attrs.items():
            setattr(self, k, v)
    
class Observable(Base,TObservable, Generic[T]):
    def __init__(self, value: T):
        self.value: T = value
        self.callbacks: Set[TSubscriber] = set()
    def __repr__(self) -> str: return f"{self.__class__.__name__}({str(self.value)}, |cbs|= {len(self.callbacks)})"
    def subscribe(self, subscriber: TSubscriber) -> TUnsubscriber:
        self.callbacks.add(subscriber)
        e = Event(source=repr(self), type="UPDATE", payload = self.value)
        if (len(self.callbacks) ==1): e.type = "START" # notify subscriber it is the only callback
        subscriber(e) # notify subscriber of the current value
        
        def unsubscribe()-> None: 
            if (len(self.callbacks) == 1):
                e = Event(source=repr(self), type="STOP", payload = self.value)
                self.emits(e)
            # reference to unsubscribe function can be kept by the subscriber
            # and invoked later to unsubscribe
            self.callbacks.remove(subscriber) if subscriber in self.callbacks else noop
        return unsubscribe
    
    def emits(self, msg: Event) -> None:
        for callback in self.callbacks:
           callback(msg)

In [65]:
class Base:
    def __str__(self) -> str: return repr(self)
    def __repr__(self) -> str: return f"{self.__class__.__name__}({str(self.__dict__)})"
    
@dataclass
class Event:
    source: str
    type: str
    payload: Any = None
    
    def __init__(self, source: str, **attrs: Any) -> None:
        self.source = source
        for k, v in attrs.items():
            setattr(self, k, v)
    
class Observable(Base,TObservable, Generic[T]):
    def __init__(self, value: T):
        self.value: T = value
        self.callbacks: Set[TSubscriber] = set()
    def __repr__(self) -> str: return f"{self.__class__.__name__}({str(self.value)}, |cbs|= {len(self.callbacks)})"
    def subscribe(self, subscriber: TSubscriber) -> TUnsubscriber:
        self.callbacks.add(subscriber)
        e = Event(source=repr(self), type="UPDATE", payload = self.value)
        if (len(self.callbacks) ==1): e.type = "START" # notify subscriber it is the only callback
        subscriber(e) # notify subscriber of the current value
        
        def unsubscribe()-> None: 
            if (len(self.callbacks) == 1):
                e = Event(source=repr(self), type="STOP", payload = self.value)
                self.emits(e)
            # reference to unsubscribe function can be kept by the subscriber
            # and invoked later to unsubscribe
            self.callbacks.remove(subscriber) if subscriber in self.callbacks else noop
        return unsubscribe
    def emits(self, msg: Event) -> None:
        for callback in self.callbacks:
           callback(msg)
    def get(self) -> T: return self.value
    def update(self, new_value: T) -> None:
        if self.value == new_value: return None
        self.value = new_value
        e = Event(source=repr(self), type="UPDATE", payload = self.value)
        self.emits(e)


In [66]:
user = Observable({"name": "John", "age": 42})

In [70]:
user.update(user.get()|{'age': 31})

Event(source="Observable({'name': 'John', 'age': 31}, |cbs|= 1)", type='UPDATE', payload={'name': 'John', 'age': 31})


In [63]:
user

Observable({'name': 'John', 'age': 37}, |cbs|= 0)

In [67]:
user.subscribe(lambda msg: print(msg))

Event(source="Observable({'name': 'John', 'age': 42}, |cbs|= 1)", type='START', payload={'name': 'John', 'age': 42})


<function __main__.Observable.subscribe.<locals>.unsubscribe() -> 'None'>

In [73]:
class Callback(Generic[T]):
    def __init__(self, observes: Sequence[TObservable], fn: Callable[[T], None]) -> None:
        self.fn = fn
        self.observes = observes
        self.subscriptions: Set[TUnsubscriber] = set()
        for observer in observes: 
            unsubscribe = observer.subscribe(lambda msg: self.__call__(msg))
            self.subscriptions.add(unsubscribe)
    def __call__(self, msg: Event) -> None: 
        actions = {
            "START": noop,
            "UPDATE": self.fn,
            "STOP": noop
        }
        actions[msg.type](msg.payload)

In [74]:
user.update(user.get()|{'age': 22})

Event(source="Observable({'name': 'John', 'age': 22}, |cbs|= 2)", type='UPDATE', payload={'name': 'John', 'age': 22})
{'name': 'John', 'age': 22}


In [75]:
log = Callback([user], print)

{'name': 'John', 'age': 22}


In [76]:
user.update(user.get()|{'age': 33})

Event(source="Observable({'name': 'John', 'age': 33}, |cbs|= 3)", type='UPDATE', payload={'name': 'John', 'age': 33})
{'name': 'John', 'age': 33}
{'name': 'John', 'age': 33}


In [56]:
def up(val): return val+1
user.subscribe()

SyntaxError: invalid syntax (891648103.py, line 1)

In [ ]:
#| export

T = TypeVar("T")
TEvent = TypeVar('TEvent', bound=object) 
THandler = Callable[[TEvent], None] 
TPromise =  Callable[[T], Tuple[THandler, THandler]]
TSubscriber = Union[Callable[[T], None],TPromise] # action to invoke when the store changes
TUnsubscriber = Callable[[], None] # action to invoke upon graceful termination of the subscription
TUpdater = Callable[[T, TEvent], T] # action to invoke to update the store, aka reducer
noop = lambda: None

class TObserver(Protocol):
    def __call__(self, msg: TEvent) -> None: ... # type: ignore

class TObservable(Protocol):
    def subscribe(self, subscriber: TSubscriber) -> TUnsubscriber: ...
    # def emits(self, msg: Any) -> None: ...

TSubject = Union[TObservable, TObserver]

In [ ]:

# class Store(TObservable):
#     def subscribe(self, subscriber: TSubscriber) -> TUnsubscriber:
#         return noop
#     def emits(self, msg: Any) -> None:
#         pass

# class Event(object):
#     pass

# class Observable(object):
#     def __init__(self):
#         self.callbacks = []
#     def subscribe(self, callback):
#         self.callbacks.append(callback)
#     def fire(self, **attrs):
#         e = Event()
#         e.source = self
#         for k, v in attrs.items():
#             setattr(e, k, v)
#         for fn in self.callbacks:
#             fn(e)
    
# class Readable(Generic[T]):
#     def __init__(self, value: T, start: Optional[TStartStop]=None) -> None:
#         self.value: T = value
#         self.start: Optional[TStartStop] = start
#         self.subscribers: set[TSubscriber] = set() # callback list

#     def __getattr__(self, name):
#         return getattr(self.value, name)

#     def __repr__(self) -> str:
#         return f"{self.__class__.__name__}({self.value!r})"

#     def __str__(self) -> str: return str(self.value)

#     def get(self) -> T: return self.value

#     def _set(self, new_value: T) -> None:
#         if new_value != self.value: self.value = new_value
#         for subscriber in self.subscribers: subscriber(self.value)

#     def subscribe(self, subscriber: TSubscriber) -> TUnsubscriber:
#         self.subscribers.add(subscriber)
#         subscriber(self.value)
#         if (len(self.subscribers) == 1):
#              if self.start is not None: 
#                 self.stop: Union[TUnsubscriber, None] =  self.start(self._set) 
#         def unsubscribe()-> None:
#             self.subscribers.remove(subscriber)
#         if isinstance(subscriber, Subscriber): subscriber.add_subscription(unsubscribe)
#         return unsubscribe

# class Writable(Readable[T]):

#     def set(self, new_value: T) -> None:
#         self._set(new_value)
    
#     def update(self, fn: TUpdater) -> None: self.set(fn(self.value))


# class Subscriber(Generic[T]):
#     """ Represents a subscriber (a callback) to a store (an observable)."""
#     def __init__(self, fn: Callable[[T], None], observable: Optional[Union[Readable[T], Writable[T]]]) -> None:
#         self.fn = fn
#         self.subscriptions: Set[Callable[[], None]] = set()

#     def __call__(self, value: T) -> None:
#         self.fn(value)

#     def __eq__(self, other: Callable[[T], None]) -> bool:
#         return self.fn == other.fn

#     def __hash__(self) -> int:
#         return hash(self.fn)
    
#     def __del__(self)-> None:
#         for unsubscribe in self.subscriptions: unsubscribe()
        
#     def add_subscription(self, unsubscribe: Callable[[], None])-> None:
#         self.subscriptions.add(unsubscribe)
    


In [ ]:
Stores = Union[Readable[T], Sequence[Readable[T]]]
Observable = Union[Readable[T], Writable[T]]

### Export

In [ ]:
nbdev.nbdev_export()

In [ ]:
# #| export

# def writable(value: T) -> Writable[T]:
#     """ Create a writable store with a given value that allows both updating and reading by subscription."""

    
#     def set(new_value: T) -> None:
#         nonlocal value
#         if new_value != value: value = new_value
#         for subscriber in subscriber_queue: subscriber(value)

#     def update(fn: Updater[T]) -> None: set(fn(value))

#     def subscribe(subscriber: Subscriber[T]) -> Unsubscriber:
#         subscriber_queue.append(subscriber)
#         subscriber(value)

#         def unsubscribe() -> None:
#             subscriber_queue.remove(subscriber)

#         return unsubscribe

#     ret = Writable() # type: ignore
#     ret.set = set # type: ignore
#     ret.update = update # type: ignore
#     ret.subscribe = subscribe

#     return ret

In [ ]:
# #| export

# def readable(value: T) -> Readable[T]:
#     ret = Readable()
#     ret.subscribe = writable(value).subscribe
#     return ret

In [ ]:
# class Store:
#     def __init__(self, value):
#         self.value = value
#         self.subscribers = []
#     def subscribe(self, subscriber):
#         self.subscribers.append(subscriber)
#         subscriber(self.value)
#         return lambda: self.subscribers.remove(subscriber)
#     def set(self, value):
#         if value != self.value:
#             self.value = value
#             for subscriber in self.subscribers:
#                 subscriber(value)
#     def update(self, updater):
#         self.set(updater(self.value))